In [1]:
import os, sys
import time
import backtrader as bt
import logging
import argparse
import pandas as pd
import quantstats
import numpy as np
from enum import Enum
from datetime import datetime
from MyStrategy import MyStrategy  # 引用策略
from MyStrategy import MyCommission  # 引用合约信息
from my_tradeanalyzer import My_TradeAnalyzer  # 自定义分析器

G_COUNT = 0  # jupyter运行次数统计
G_CEREBRO = None  # 大脑引擎
G_RESULT_ONE = None  # 回测大脑返回
G_RESULTS_OPT = None  # 参数调优大脑返回
res_df = None  # 筛选后的参数优化结果


In [5]:

def parse_args(pargs=None):
    parser = argparse.ArgumentParser(
        formatter_class=argparse.ArgumentDefaultsHelpFormatter,
        description='Sample for Order Target')

    parser.add_argument('--data', required=False,
                        default=G_FILE_PATH,
                        help='Specific data to be read in')
    parser.add_argument('--dtformat', required=False, default=G_DT_DTFORMAT,
                        help='Ending date in data datetime format')
    parser.add_argument('--fromdate', required=False, default=G_DT_START,
                        help='Starting date in `dtformat` format')
    parser.add_argument('--todate', required=False, default=G_DT_END,
                        help='Ending date in `dtformat` format')
    parser.add_argument('--timeframe', required=False, default=G_DT_TIMEFRAME,
                        choices=['minutes', 'daily', 'weekly', 'monthly'],
                        help='重新采样到的时间范围')
    parser.add_argument('--compression', required=False, type=int, default=G_DT_COMPRESSION, help='将 n 条压缩为 1, 最小周期为原数据周期')
    parser.add_argument('--key', required=False, type=list, default=G_P_PARAM['key'] if 'key' in G_P_PARAM else None, help='当穿越关键价格后加仓限制，列表类型 [price1, price2]'),
    parser.add_argument('--rsp', required=False, type=list, default=G_P_PARAM['rsp'] if 'rsp' in G_P_PARAM else None, help='--rsp 盈亏千分比'),
    parser.add_argument('--rpp', required=False, type=list, default=G_P_PARAM['rpp'] if 'rpp' in G_P_PARAM else None, help='--rpp 盈利千分比'),
    parser.add_argument('--spp', required=False, type=list, default=G_P_PARAM['spp'] if 'spp' in G_P_PARAM else None, help='--spp 亏损千分比'),
    parser.add_argument('--ojk', required=False, type=list, default=G_P_PARAM['ojk'] if 'ojk' in G_P_PARAM else None, help='--ojk 订单间隔bar周期数'),
    parser.add_argument('--opts', required=False, type=bool, default=G_OPTS, help='是否策略优化')
    parser.add_argument('--quantstats', required=False, type=int, default=G_QUANTSTATS, help='是否使用 quantstats 分析测试结果')
    parser.add_argument('--maxcpus', '-m', type=int, required=False, default=15, help=('Number of CPUs to use in the optimization'
                                                                                       '\n  - 0 (default): 使用所有可用的 CPU\n   - 1 -> n: 使用尽可能多的指定\n'))
    parser.add_argument('--no-optdatas', action='store_true', required=False, help='优化中不优化数据预加载')
    parser.add_argument('--no-optreturn', action='store_true', required=False,
                        help='不要优化返回值以节省时间,这避免了回传大量生成的数据，例如指标在回溯测试期间生成的值')

    # Plot options
    parser.add_argument('--plot', '-p', nargs='?', required=False,
                        metavar='kwargs', const=True, default=G_PLOT,
                        help=('绘制应用传递的任何 kwargs 的读取数据\n'
                              '\n''例如:\n''\n''  --plot style="candle" (to plot candles)\n'))
    parser.add_argument("-f", "--file", default="file")  # 接收这个-f参数
    if pargs is not None:
        return parser.parse_args(pargs)

    return parser.parse_args()


def runstrat(args=None):
    global G_CEREBRO, G_RESULT_ONE, G_RESULTS_OPT, res_df  # 申明要使用全局变量
    args = parse_args(args)
    kwargs = dict()  # 参数字典
    kwargs['test_kwargs'] = dict()  # 回测参数字典
    kwargs['opts_kwargs'] = dict()  # 优化参数字典
    kwargs['log_kwargs'] = dict()  # 日志参数字典

    file_path_abs = dt_start = dt_end = dt_format = dt_dtformat = dt_tmformat = ""

    if args.dtformat is not None:
        dt_format = args.dtformat
        dt_dtformat = dt_format[:dt_format.find('%d') + len('%d')]
        dt_tmformat = dt_format[dt_format.find('%H'):]
        # dkwargs['dtformat'] = dt_format
        # dkwargs['tmformat'] = dt_tmformat
    if args.fromdate is not None:
        dt_start = datetime.strptime(args.fromdate, dt_dtformat).date()
        # dkwargs['fromdate'] = dt_start
    if args.todate is not None:
        dt_end = datetime.strptime(args.todate, dt_dtformat).date()
        # dkwargs['todate'] = dt_end
    if args.data is not None:
        file_path = args.data
        myQuant_ROOT = os.getcwd()[:os.getcwd().find("bt_backtrader\\") + len("bt_backtrader\\")]  # 获取项目中相对根路径
        file_path_abs = os.path.join(myQuant_ROOT, file_path)  # 文件路径
        print(file_path_abs)
        # print("dt_format:", dt_format, "dt_start:", datetime.strftime(dt_start, "%Y-%m-%d"), "dt_end:", datetime.strftime(dt_end, "%Y-%m-%d"))
        if not os.path.exists(file_path_abs):
            raise Exception("数据源文件未找到！" + file_path_abs)
    if args.rsp is not None:
        rsp = args.rsp
        kwargs['test_kwargs']['rsp'] = rsp
    if args.rpp is not None:
        rpp = args.rpp
        kwargs['test_kwargs']['rpp'] = rpp
    if args.spp is not None:
        spp = args.spp
        kwargs['test_kwargs']['spp'] = spp
    if args.ojk is not None:
        ojk = args.ojk
        kwargs['test_kwargs']['ojk'] = ojk
    if args.key is not None:
        key = args.key
        kwargs['test_kwargs']['key'] = key

    # 加载数据
    df_data = pd.read_csv(filepath_or_buffer=file_path_abs,
                          # parse_dates={'datetime': ['date', 'time']},  # 日期和时间分开的情况
                          parse_dates=['datetime'],
                          index_col='datetime',
                          infer_datetime_format=True,
                          )

    # df.sort_values(by=["datetime"], ascending=True, inplace=True)  # 按日期先后排序
    # df.sort_values(by=["date", "time"], ascending=True, inplace=True)  # 按日期时间列先后排序

    # df.index = pd.to_datetime(df.date + ' ' + df.time, format=dt_format)  # 方式1: 将日期列和时间合并后转换成日期类型,并设置成列索引
    # df.index = pd.to_datetime(df.date.astype(str) + ' ' + df.time.astype(str), format=dt_format)  # 方式2: 将日期列和时间合并后转换成日期类型,并设置成列索引
    # df.index = pd.to_datetime(df['date'] + ' ' + df['time'], format=dt_format)  # 方式3: 将日期列和时间合并后转换成日期类型,并设置成列索引
    # df.index = pd.to_datetime(df['date'], format=dt_dtformat) + pd.to_timedelta(df['time'])  # 方式4: 将日期列和时间合并后转换成日期类型,并设置成列索引
    # df.index = pd.to_datetime(df.pop('date')) + pd.to_timedelta(df.pop('time'))  # 方式5: 将日期列和时间合并后转换成日期类型,并设置成列索引
    # df.index = pd.to_datetime(df['date'].str.cat(df['time'], sep=' '), format=dt_format)  # 方式6: 将日期列和时间合并后转换成日期类型,并设置成列索引
    # 截取时间段内样本数据
    df_data = df_data[dt_start:dt_end]
    # 增加一列openinterest
    df_data['openinterest'] = 0.00
    # 取出特定的列
    df_data = df_data[['open', 'high', 'low', 'close', 'volume']]
    # 列名修改成指定的
    df_data.rename(columns={"volume": "vol"}, inplace=True)

    tframes = dict(
        minutes=bt.TimeFrame.Minutes,
        daily=bt.TimeFrame.Days,
        weekly=bt.TimeFrame.Weeks,
        monthly=bt.TimeFrame.Months)

    # 使用pandas数据源创建交易数据集
    # data = bt.feeds.GenericCSVData(dataname=file_path, **dkwargs)
    data = (bt.feeds.PandasData(dataname=df_data, fromdate=dt_start, todate=dt_end))
    # 重采样到更大时间框架
    if args.timeframe and args.compression:
        data.resample(timeframe=tframes[args.timeframe], compression=args.compression)

    G_CEREBRO = cerebro = bt.Cerebro(stdstats=False)

    cerebro.adddata(data)
    # 设置投资金额1000000
    cerebro.broker.setcash(1000000)

    # # <editor-fold desc="折叠代码:交易手续费设置一">
    # cerebro.broker.setcommission(
    #     # 交易手续费，根据margin取值情况区分是百分比手续费还是固定手续费
    #     commission=0.0015,
    #     # commission=4,
    #     # 期货保证金，决定着交易费用的类型,只有在stocklike=False时起作用
    #     margin=0,
    #     # 乘数，盈亏会按该乘数进行放大
    #     mult=10.0,
    #     # 交易费用计算方式，取值有：
    #     # 1.CommInfoBase.COMM_PERC 百分比费用
    #     # 2.CommInfoBase.COMM_FIXED 固定费用
    #     # 3.None 根据 margin 取值来确定类型
    #     commtype=bt.CommInfoBase.COMM_PERC,
    #     # 当交易费用处于百分比模式下时，commission 是否为 % 形式
    #     # True，表示不以 % 为单位，0.XX 形式；False，表示以 % 为单位，XX% 形式
    #     percabs=True,
    #     # 是否为股票模式，该模式通常由margin和commtype参数决定
    #     # margin=None或COMM_PERC模式时，就会stocklike=True，对应股票手续费；
    #     # margin设置了取值或COMM_FIXED模式时,就会stocklike=False，对应期货手续费
    #     stocklike=False,
    #     # 计算持有的空头头寸的年化利息
    #     # days * price * abs(size) * (interest / 365)
    #     interest=0.0,
    #     # 计算持有的多头头寸的年化利息
    #     interest_long=False,
    #     # 杠杆比率，交易时按该杠杆调整所需现金
    #     leverage=1.0,
    #     # 自动计算保证金
    #     # 如果 False, 则通过margin参数确定保证金
    #     # 如果automargin<0, 通过mult*price确定保证金
    #     # 如果automargin>0, 如果 automargin*price确定保证金 automargin=mult*margin
    #     automargin=10 * 0.13,
    #     # 交易费用设置作用的数据集(也就是作用的标的)
    #     # 如果取值为None，则默认作用于所有数据集(也就是作用于所有assets)
    #     name=None
    # )
    # # </editor-fold>
    pass
    # <editor-fold desc="折叠代码:交易手续费设置方式二">
    comm = {
        'comm_sqrb': MyCommission(commtype=bt.CommInfoBase.COMM_PERC, commission=0.00015, margin_rate=0.13, mult=10.0),  # 螺纹钢合约信息
        'comm_zjif': MyCommission(commtype=bt.CommInfoBase.COMM_PERC, commission=0.00050, margin_rate=0.23, mult=300.0),  # 股指合约信息
        'comm_zqcf': MyCommission(commtype=bt.CommInfoBase.COMM_FIXED, commission=2.4, margin_rate=0.10, mult=10.0),  # 玉米合约信息
    }
    # 添加进 broker
    cerebro.broker.addcommissioninfo(comm[G_COMM], name=None)  # name 用于指定该交易费用函数适用的标的,未指定将适用所有标的
    # </editor-fold>

    strats = None
    result_one = G_RESULT_ONE
    results_opt = G_RESULTS_OPT
    file_name = '{:}_{:}_{:}_{:}_{:}'.format(G_FILE_PATH[:12], (str(G_DT_COMPRESSION) + (G_DT_TIMEFRAME[:1])), G_DT_START, G_DT_END, (str(G_P_PARAM).replace(' ', '').replace('\'', '').replace(':', '=')), )
    file_name = (file_name + ('_rs' if G_OPTS_RESULT else file_name))

    # 参数调优
    if args.opts:
        kwargs['opts_kwargs'] = G_P_PARAM  # 优化参数字典
        kwargs['opts_path'] = (file_name + '_opt.csv')  # 优化结果保存路径
        print('dt_start:', dt_start, 'dt_end:', dt_end)
        print('opts_kwargs:', kwargs['opts_kwargs'])
        # clock the start of the process
        tstart = time.perf_counter()
        # 为Cerebro引擎添加策略, 优化策略
        strats = cerebro.optstrategy(MyStrategy, **kwargs['opts_kwargs'])
        # 添加分析指标
        cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='timeReturn', timeframe=bt.TimeFrame.Years)  # 此分析器通过查看时间范围的开始和结束来计算回报
        cerebro.addanalyzer(bt.analyzers.Returns, _name="returns")  # 使用对数方法计算的总回报、平均回报、复合回报和年化回报
        cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyFolio')  # 添加PyFolio
        cerebro.addanalyzer(bt.analyzers.DrawDown, _name="drawdown")  # 回撤
        cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name="sharpe")  # 夏普率
        cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name="tradeAnalyzer")  # 提供有关平仓交易的统计信息（也保留未平仓交易的数量）
        cerebro.addanalyzer(My_TradeAnalyzer, _name="my_tradeAnalyzer")  # 自定义平仓交易的统计信息
        # Run over everything
        if G_OPTS_RESULT and results_opt:  # 是否使用上次参数优化结果
            G_RESULTS_OPT = results_opt
            print("--------------- 上次参数优化结果 -----------------")
        else:
            G_RESULTS_OPT = results_opt = cerebro.run(
                maxcpus=args.maxcpus,
                optdatas=not args.no_optdatas,  # optdatas（默认值：True)如果和优化（以及系统可以和使用），数据预加载将只在主进程中完成一次，以节省时间和资源。
                optreturn=not args.no_optreturn,  # optreturn（默认值：True)如果优化结果不是完整的对象（以及所有数据、指标、观察器等），而是具有以下属性的对象 在大多数情况下，只有分析器和哪些参数是评估策略性能所需的东西。如果需要对（例如）指标的生成值进行详细分析，请将其关闭
                # optreturn=False,
                # stdstats=False,
            )
            print("--------------- 参数优化结果 -----------------")
        # clock the end of the process
        tend = time.perf_counter()
        # print out the results_opt
        print('Time used:', str(tend - tstart))
        # 每个策略实例的结果以列表的形式保存在列表中。优化运行模式下，返回值是列表的列表,内列表只含一个元素，即策略实例
        res_list = [[]]
        res_timereturn_title = []  # 列标题

        timeReturn = results_opt[0][0].analyzers.timeReturn.get_analysis()  # timeReturn 分析引用
        for k, v in timeReturn.items():
            res_timereturn_title.append('{:%Y-%m}'.format(k))
        columns = ['rsp', 'rpp', 'spp', 'ojk', 'total', 'sharpe', 'rtot%', 'py_rt%', 'won%', 'rnorm%', 'maxDD%', 'comm%', 'pnl_net']
        columns_all = columns.copy()
        columns_all.extend(res_timereturn_title)  # 将列标题添加到columns列表尾部
        res_df = pd.DataFrame(columns=columns_all)

        for i, x in enumerate(results_opt):
            trade = x[0].analyzers.tradeAnalyzer.get_analysis()  # 交易分析引用
            my_trade = x[0].analyzers.my_tradeAnalyzer.get_analysis()  # 交易分析引用
            returns = x[0].analyzers.returns.get_analysis()  # 回报分析引用
            pyFolio = x[0].analyzers.pyFolio.get_analysis()  # pyFolio分析引用
            drawdown = x[0].analyzers.drawdown.get_analysis()  # 回撤分析引用
            sharpe = x[0].analyzers.sharpe.get_analysis()  # sharpe分析引用
            timeReturn = x[0].analyzers.timeReturn.get_analysis()  # timeReturn 分析引用

            if trade['total']['total'] == 0:
                continue  # 忽略交易次数为0 的数据

            returns_rort_ = returns['rtot'] * 100  # 总复合回报
            pyFolio_returns_ = sum(pyFolio['returns'].values()) * 100  # pyFolio总复合回报
            returns_rnorm100_ = returns['rnorm100'] * 100  # 年化归一化回报
            trade_won_ = (trade['won']['total']) if 'won' in trade else 0  # 总盈利次数
            trade_lost_ = (trade['lost']['total']) if 'lost' in trade else 0  # 总亏损次数
            trade_total_ = trade['total']['total']  # 交易次数
            trade_win_rate = (trade_won_ / trade_total_) * 100  # 胜率
            drawdown_ = drawdown['max']['drawdown']
            sharpe_ = sharpe['sharperatio']
            trade_pnl_total_ = (trade['pnl']['gross']['total']) if 'pnl' in trade else 0  # 总盈亏
            trade_pnl_net_ = (trade['pnl']['net']['total']) if 'pnl' in trade else 0  # 总盈亏-手续费
            trade_pnl_comm_ = abs(trade_pnl_total_ - trade_pnl_net_)  # 手续费
            trade_comm_net_p = ((trade_pnl_comm_ / trade_pnl_net_) * 100) if trade_pnl_net_ != 0 else 0  # 手续费占比净盈亏百分比

            row = ({
                'rsp': x[0].p.rsp,  # 参数
                'rpp': x[0].p.rpp,  # 参数
                'spp': x[0].p.spp,  # 参数
                'ojk': x[0].p.ojk,  # 参数
                'total': '{:0>4d}'.format(trade_total_),  # 交易次数
                'sharpe': (sharpe_ if sharpe_ else 0),  # 夏普率
                'rtot%': returns_rort_,  # 总复合回报
                'py_rt%': pyFolio_returns_,  # pyFolio总复合回报
                'won%': trade_win_rate,  # 胜率
                'rnorm%': returns_rnorm100_,  # 年化归一化回报
                'maxDD%': (drawdown_ if drawdown_ else 0),  # 最大回撤
                'comm%': trade_comm_net_p,  # 手续费占比净盈亏百分比
                'pnl_net': '{:8.2f}'.format(trade_pnl_net_),  # 总盈亏余额含手续费
            })
            for k, v in timeReturn.items():  # 把timeReturn统计的月度或年度复合回报添加在后面 # 月度或年度复合回报,由参数timeframe=bt.TimeFrame.Months控制
                row['{:%Y-%m}'.format(k)] = v
            res_df = res_df.append(row, ignore_index=True)
        res_df[['rpp', 'spp', 'ojk', 'total']] = res_df[['rpp', 'spp', 'ojk', 'total']].astype(int)  # 转换指定类数据类型
        if not bool(res_df.notnull):
            print('回测数据不存在,退出')
        if not ('rpp' in G_P_PARAM or 'spp' in G_P_PARAM):
            # 删除未优化的参数列
            res_df = res_df.drop(labels=['rpp', 'spp'], axis=1)
            columns.remove('rpp')
            columns.remove('spp')
        if not ('ojk' in G_P_PARAM):
            res_df = res_df.drop(labels=['ojk'], axis=1)
            columns.remove('ojk')
        if not ('rsp' in G_P_PARAM):
            res_df = res_df.drop(labels=['rsp'], axis=1)
            columns.remove('rsp')

        res_df = res_df.dropna(how='any', axis=0)  # 删除所有带NaN的行
        # res_df[['rpp', 'spp', 'ojk', 'total']] = res_df[['rpp', 'spp', 'ojk', 'total']].apply(pd.to_numeric, downcast='signed', axis=1)  # 转换指定列数据类型为整形
        res_df[['rtot%', 'pnl_net']] = res_df[['rtot%', 'pnl_net']].apply(pd.to_numeric, errors='ignore', axis=1)

        res_df.sort_values(by=['pnl_net', 'rtot%'], ascending=False, inplace=True)  # 按累计盈亏和总复合回报排序
        # res_df.reset_index(drop=True, inplace=True)  # 重设索引id,删除旧索引,替换新索引
        res_df.index.name = 'id'  # 设置索引名称
        pd.set_option('precision', 3)  # 显示小数点后的位数
        pd.set_option('display.min_rows', 300)  # 确定显示的部分有多少行
        pd.set_option('display.max_rows', 300)  # 确定显示的部分有多少行
        pd.set_option('display.max_columns', 20)  # 确定显示的部分有多少列
        pd.set_option('display.float_format', '{:,.2f}'.format)  # 逗号格式化大值数字,设置数据精度
        pd.set_option('expand_frame_repr', False)  # True就是可以换行显示。设置成False的时候不允许换行
        pd.set_option('display.width', 180)  # 设置打印宽度(**重要**)

        res_df[res_df.columns[:5]].info()  # 显示前几列的数据类型
        G_RESULT_ONE = result_one = results_opt[res_df.index[0]]  # 返回第一个参数测试结果
        opts_path = kwargs['opts_path']
        print(opts_path)  # 打印文件路径
        print(res_df.loc[:, :'pnl_net'])  # 显示 开始列到'pnl_net'列的 参数优化结果
        res_df.to_csv(opts_path, sep='\t', float_format='%.2f')  # 保存分析数据到文件
    # 回测分析
    if not args.opts:
        test_kwargs = kwargs['test_kwargs']
        # 回测日志参数
        test_kwargs['log_kwargs'] = dict(log_name='交易日志',
                                         log_print=G_P_LOG_PRINT,  # 是否打印日志到控制台
                                         log_save=G_P_LOG_FILE,  # 是否保存日志到文件
                                         log_path=(file_name + '_log.txt'),  # 日志保存文件路径
                                         )

        log_kwargs = test_kwargs['log_kwargs']
        print('dt_start:', dt_start, 'dt_end:', dt_end)
        print('test_kwargs:', test_kwargs)  # 回测参数
        # print('log_kwargs:', log_kwargs)  # 日志参数
        # 添加观测器,绘制时显示
        cerebro.addobserver(bt.observers.Broker)
        cerebro.addobserver(bt.observers.Trades)
        cerebro.addobserver(bt.observers.BuySell)
        # cerebro.addobserver(bt.observers.TimeReturn)
        cerebro.addobserver(bt.observers.DrawDown)
        # 添加分析指标
        cerebro.addanalyzer(bt.analyzers.AnnualReturn, _name='annualReturn')  # 返回年初至年末的年度收益率
        cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawDown')  # 计算最大回撤相关指标
        cerebro.addanalyzer(bt.analyzers.Returns, _name='returns', tann=252)  # 计算年化收益：日度收益
        cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyFolio')  # 添加PyFolio
        cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpeRatio', timeframe=bt.TimeFrame.Days, annualize=True, riskfreerate=0)  # 计算年化夏普比率：日度收益
        cerebro.addanalyzer(bt.analyzers.SharpeRatio_A, _name='sharpeRatio_A')
        cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='timeReturn', )  # 添加收益率时序
        cerebro.addanalyzer(My_TradeAnalyzer, _name="my_tradeAnalyzer")  # 自定义平仓交易的统计信息

        # 添加策略和参数
        cerebro.addstrategy(MyStrategy, **test_kwargs)
        logger = None
        if G_P_LOG_PRINT or G_P_LOG_FILE:
            logger = logger_config(log_path=str(log_kwargs['log_path']), log_name=log_kwargs['log_name'])

        # 引擎运行前打印期出资金
        print('组合期初资金: %s' % format(cerebro.broker.getvalue(), ',.2f'))
        # 启动回测
        G_RESULT_ONE = result_one = cerebro.run()
        # 引擎运行后打期末资金
        print('组合期末资金: %s' % format(cerebro.broker.getvalue(), ',.2f'))
        # 提取结果
        print("\n--------------- 累计收益率 -----------------")
        annualReturn = result_one[0].analyzers.annualReturn.get_analysis()
        pyFolio = result_one[0].analyzers.pyFolio.get_analysis()
        print(" Cumulative Return: {:.2f}".format(sum(annualReturn.values())))
        print(" pyFolio Cumulative Return%: {:,.2f}".format(sum(pyFolio['returns'].values()) * 100))
        print("\n--------------- 年度收益率 -----------------")
        # print(' 收益率k,v', get_analysis.items())
        for k, v in annualReturn.items():
            print((" [{:},{:.2f}]" if isinstance(v, float) else " [{:},{:}]").format(k, v), end='')
        print("\n--------------- 最大回撤 -----------------")
        drawDown = result_one[0].analyzers.drawDown.get_analysis()
        for k, v in drawDown.items():
            if not isinstance(v, dict):
                t = (" [{:},{:.2f}]" if isinstance(v, float) else " [{:},{:}]").format(k, v)
                print(t, end='')
            else:
                for kk, vv in v.items():
                    t = (" [{:},{:.2f}]" if isinstance(vv, float) else " [{:},{:}]").format(kk, vv)
                    print(t, end='')
        print("\n--------------- 年化收益：日度收益 -----------------")
        an_returns = result_one[0].analyzers.returns.get_analysis()
        for k, v in an_returns.items():
            print((" [{:},{:.2f}]" if isinstance(v, float) else " [{:},{:}]").format(k, v), end='')
        print("\n--------------- 年化夏普比率：日度收益 -----------------")
        sharpeRatio = (result_one[0].analyzers.sharpeRatio.get_analysis())
        for k, v in sharpeRatio.items():
            print((" [{:},{:.2f}]" if isinstance(v, float) else " [{:},{:}]").format(k, v), end='')
        print("\n--------------- SharpeRatio_A -----------------")
        sharpeRatio_A = result_one[0].analyzers.sharpeRatio_A.get_analysis()
        for k, v in sharpeRatio_A.items():
            print((" [{:},{:.2f}]" if isinstance(v, float) else " [{:},{:}]").format(k, v), end='')
        print("\n--------------- test end -----------------")
        # 在记录日志之后移除句柄
        if G_P_LOG_PRINT or G_P_LOG_FILE:
            if logger.streamHandler:
                logger.streamHandler.close()
            if logger.fileHandler:
                logger.fileHandler.close()
            logging.shutdown()  # 关闭日志系统
    # 绘图
    if args.plot and not args.opts:
        pkwargs = dict(style='bar')
        if args.plot is not True:  # evals to True but is not True
            npkwargs = eval('dict(' + args.plot + ')')  # args were passed
            pkwargs.update(npkwargs)

        cerebro.plot(volume=False, **pkwargs)  # 绘图BT观察器结果

        # 结合pyfolio工具 计算并绘制收益评价指标
        import pyfolio as pf
        # 绘制图形
        import matplotlib.pyplot as plt
        plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
        import matplotlib.ticker as ticker  # 导入设置坐标轴的模块
        # plt.style.use('seaborn')
        plt.style.use('dark_background')

        # 提取收益序列
        pnl = pd.Series(result_one[0].analyzers.timeReturn.get_analysis())
        # 计算累计收益
        cumulative = (pnl + 1).cumprod()
        # 计算回撤序列
        max_return = cumulative.cummax()
        drawdown = (cumulative - max_return) / max_return
        # 按年统计收益指标
        perf_stats_year = pnl.groupby(pnl.index.to_period('y')).apply(lambda data: pf.timeseries.perf_stats(data)).unstack()
        # 统计所有时间段的收益指标
        perf_stats_all = pf.timeseries.perf_stats(pnl).to_frame(name='all')
        perf_stats = pd.concat([perf_stats_year, perf_stats_all.T], axis=0)
        perf_stats_ = round(perf_stats, 4).reset_index()

        fig, (ax0, ax1) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [3, 4]}, figsize=(24, 16))
        cols_names = ['date', 'Annual\nreturn', 'Cumulative\nreturns', 'Annual\n volatility',
                      'Sharpe\nratio', 'Calmar\nratio', 'Stability', 'Max\ndrawdown',
                      'Omega\nratio', 'Sortino\nratio', 'Skew', 'Kurtosis', 'Tail\nratio',
                      'Daily value\nat risk']
        # 绘制表格
        ax0.set_axis_off()  # 除去坐标轴
        table = ax0.table(cellText=perf_stats_.values,
                          bbox=(0, 0, 1, 1),  # 设置表格位置， (x0, y0, width, height)
                          rowLoc='left',  # 行标题居中
                          cellLoc='left',
                          colLabels=cols_names,  # 设置列标题
                          colLoc='left',  # 列标题居中
                          edges='open'  # 不显示表格边框
                          )
        table.set_fontsize(13)

        # 绘制累计收益曲线
        ax2 = ax1.twinx()
        ax1.yaxis.set_ticks_position('right')  # 将回撤曲线的 y 轴移至右侧
        ax2.yaxis.set_ticks_position('left')  # 将累计收益曲线的 y 轴移至左侧
        # 绘制回撤曲线
        drawdown.plot.area(ax=ax1, label='drawdown (right)', rot=0, alpha=0.3, fontsize=13, grid=False)
        # 绘制累计收益曲线
        cumulative.plot(ax=ax2, color='#F1C40F', lw=2.0, label='cumret (left)', rot=0, fontsize=13, grid=False)
        # 不然 x 轴留有空白
        ax2.set_xbound(lower=cumulative.index.min(), upper=cumulative.index.max())
        # 主轴定位器：每 5 个月显示一个日期：根据具体天数来做排版
        ax2.xaxis.set_major_locator(ticker.MultipleLocator(120))
        # 同时绘制双轴的图例
        h1, l1 = ax1.get_legend_handles_labels()
        h2, l2 = ax2.get_legend_handles_labels()

        plt.legend(h1 + h2, l1 + l2, fontsize=12, loc='upper left', ncol=1)
        fig.tight_layout()  # 规整排版
        plt.show()
        # 结合pyfolio工具 计算并绘制收益评价指标
    # 回测分析保存到文件
    if args.quantstats and not args.opts:
        # 使用quantstats 分析工具并保存到HTML文件
        result_one = G_RESULT_ONE
        portfolio_stats = result_one[0].analyzers.getbyname('pyFolio')
        returns, positions, transactions, gross_lev = portfolio_stats.get_pf_items()
        returns.index = returns.index.tz_convert(None)
        import quantstats
        # 将分析指标保存到HTML文件
        title_report = G_FILE_PATH + ' param:{:} dt:{:%H:%M:%S}'.format(str(G_P_PARAM).replace(' ', '').replace('\'', ''), datetime.now())  # 优化结果网页标题
        quantstats.reports.html(returns, output='stats.html', title=title_report)
        print("quantstats 测试分析结果已保存至目录所在文件 quantstats-tearsheet.html")
        # 使用quantstats 分析工具并保存到HTML文件


def logger_config(log_path, log_name):
    """
    配置log
    :param log_path: 输出log路径
    :param log_name: 记录中name，可随意
    :return:
    """
    '''
    logger是日志对象，handler是流处理器，console是控制台输出（没有console也可以，将不会在控制台输出，会在日志文件中输出）
    '''
    # 获取logger对象,取名
    logger = logging.getLogger(log_name)
    # 输出DEBUG及以上级别的信息，针对所有输出的第一层过滤
    logger.setLevel(level=logging.DEBUG)
    # 获取文件日志句柄并设置日志级别，第二层过滤
    logger.fileHandler = logging.FileHandler(log_path, encoding='UTF-8')
    logger.fileHandler.setLevel(logging.INFO)  # 设置文件日志输出级别 设置 INFO 时debug信息将不显示
    # 生成并设置文件日志格式
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    logger.fileHandler.setFormatter(formatter)
    # console相当于控制台输出，handler文件输出。获取流句柄并设置日志级别，第二层过滤
    logger.streamHandler = logging.StreamHandler(stream=sys.stdout)
    # logging.basicConfig(level=logging.DEBUG, stream=sys.stdout)
    logger.streamHandler.setLevel(logging.INFO)  # 设置控制台显示级别 日志级别： debug < info < warning < error < critical
    # 为logger对象添加句柄
    logger.addHandler(logger.fileHandler)
    logger.addHandler(logger.streamHandler)
    return logger


G_FILE_PATH = "datas\\ZJIF13-5m-20100416-20220427.csv"
# DT_FILE_PATH = "datas\\ZQCF13-5m-20121224-20220415.csv"
# G_FILE_PATH = "datas\\SQRB13-5m-20121224-20220330.csv"
G_DT_DTFORMAT = '%Y-%m-%d %H:%M:%S'
G_DT_START, G_DT_END = '2013-01-01', '2013-02-01'
G_COMM = 'comm_' + G_FILE_PATH.split('\\')[1][:4].lower()  # 合约信息,提前预设好 保证金,手续费率,合约乘数等
G_DT_TIMEFRAME = 'minutes'  # 重采样更大时间周期 choices=['minutes', 'daily', 'weekly', 'monthly']
G_DT_COMPRESSION = 15  # 合成bar的周期数
G_PLOT = False  # 是否绘图,可提供绘图参数:'style="candle"'
G_QUANTSTATS = True  # 是否使用 quantstats 分析测试结果
G_P_LOG_FILE = True  # 是否输出日志到文件
G_P_LOG_PRINT = False  # 是否输出日志到控制台

G_OPTS = False  # 是否参数调优
G_P_RPP = [8, True, 2, 5, 1]  # 参数[默认值,是否优化最小值,最大值,步长]
G_P_SPP = [8, True, 1, 5, 2]  # 参数[默认值,是否优化最小值,最大值,步长]
G_P_RSP = [5, False, 2, 10, 1]  # 参数[默认值,是否优化最小值,最大值,步长]
G_P_OJK = [1, False, 1, 100, 20]  # 参数[默认值,是否优化最小值,最大值,步长]
G_P_KEY = [False, 4100, 3400, 1800, 6000]  # 关键价格[是否启用, 价格1, 价格2]
G_P_PARAM = {
    'rpp': (range(G_P_RPP[2], G_P_RPP[3], G_P_RPP[4]) if G_OPTS and G_P_RPP[1] else G_P_RPP[0]),
    'spp': (range(G_P_SPP[2], G_P_SPP[3], G_P_SPP[4]) if G_OPTS and G_P_SPP[1] else G_P_SPP[0]),
    # 'rsp': (range(G_P_RSP[2], G_P_RSP[3], G_P_RSP[4]) if G_OPTS and G_P_RSP[1] else G_P_RSP[0]),
    # 'ojk': (range(G_P_OJK[2], G_P_OJK[3], G_P_OJK[4]) if G_OPTS and G_P_OJK[1] else G_P_OJK[0]),
    # 'key': G_P_KEY[1:] if G_P_KEY[0] else None
}
G_OPTS_RESULT = True  # 是否使用上次优化结果

"""-------主函数---------"""
if __name__ == '__main__':
    runstrat()

D:\study\python-demo\myQuant\bt_backtrader\datas\ZJIF13-5m-20100416-20220427.csv
dt_start: 2013-01-01 dt_end: 2013-02-01
test_kwargs: {'rpp': 8, 'spp': 8, 'log_kwargs': {'log_name': '交易日志', 'log_print': False, 'log_save': True, 'log_path': 'datas\\ZJIF13_15m_2013-01-01_2013-02-01_{rpp=8,spp=8}_rs_log.txt'}}
组合期初资金: 1,000,000.00
Fri 2013-01-04 10:30:00, notify_order:,已卖出,成交:-1,持仓:-1,Price:2550.40,Cost:175977.60,Comm:1.28,add:2530.06,exit:2570.70,open_m:2705.6,m_rate:0.23,margin:175239.30,可用资金:827231.12,持仓市值:175239.30,总资产:1,002,470.42,开仓比:-18%
Fri 2013-01-04 11:30:00, notify_order:,已卖出,成交:-1,持仓:-2,Price:2513.70,Cost:173445.30,Comm:1.26,add:2494.77,exit:2531.16,open_m:2705.6,m_rate:0.23,margin:174135.30,可用资金:655584.57,持仓市值:348270.60,总资产:1,003,855.17,开仓比:-35%
Fri 2013-01-04 13:30:00, notify_order:,已买入,成交:2,持仓:0,Price:2538.60,Cost:349422.90,Comm:2.54,add:2494.77,exit:2531.16,open_m:2705.6,m_rate:0.23,margin:175025.40,可用资金:996064.93,持仓市值:0.00,总资产:996,064.93,开仓比:10%
Fri 2013-01-04 13:30:00, n

C:\ProgramData\Anaconda3\envs\py39\lib\site-packages\seaborn\distributions.py:316: UserWarning: Dataset has 0 variance; skipping density estimate. Pass `warn_singular=False` to disable this warning.
  warnings.warn(msg, UserWarning)


quantstats 测试分析结果已保存至目录所在文件 quantstats-tearsheet.html


In [4]:
import quantstats

# 使用quantstats 分析工具并保存到HTML文件
global G_RESULT_ONE, G_RESULTS_OPT, G_COUNT  # 申明要使用全局变量
G_COUNT += 1
results_opt = G_RESULTS_OPT

id = 161
result_one_id = results_opt.index(G_RESULT_ONE)
result_one_id = result_one_id if id < 0 else id
result_one = results_opt[result_one_id - 1][0]
portfolio_stats = result_one.analyzers.getbyname('pyFolio')
returns, positions, transactions, gross_lev = portfolio_stats.get_pf_items()
returns.index = returns.index.tz_convert(None)

title = ('{:}_{:}_{:}_{:} id:{:} param:{:} d_t:{:%H:%M:%S}'
         .format(G_FILE_PATH[:12], (str(G_DT_COMPRESSION) + (G_DT_TIMEFRAME[:1])),
                 G_DT_START, G_DT_END, result_one_id, str(G_P_PARAM).replace(' ', '').replace('\'', ''), datetime.now()))
quantstats.reports.html(returns, output='stats.html', title=title)  # 将分析指标保存到HTML文件
print(title)
print("quantstats 测试分析结果已保存至目录所在文件 quantstats-tearsheet.html")
# 使用quantstats 分析工具并保存到HTML文件

datas\ZJIF13_15m_2013-01-01_2018-02-01 id:161 param:{rpp:range(2,20),spp:range(2,20)} d_t:01:25:13
quantstats 测试分析结果已保存至目录所在文件 quantstats-tearsheet.html


In [41]:
'comm_' + G_FILE_PATH.split('\\')[1][:4].lower()

'comm_sqrb'

In [2]:
# DT_FILE_PATH = "datas\\ZQCF13-5m-20121224-20220415.csv"
# G_FILE_PATH = "datas\\SQRB13-5m-20121224-20220330.csv"
G_FILE_PATH = "datas\\ZJIF13-5m-20100416-20220427.csv"
G_DT_DTFORMAT = '%Y-%m-%d %H:%M:%S'
G_DT_START, G_DT_END = '2013-01-04', '2022-04-02'
G_COMM = 'comm_' + G_FILE_PATH.split('\\')[1][:4].lower()  # 合约信息,提前预设好 保证金,手续费率,合约乘数等
G_DT_TIMEFRAME = 'minutes'  # 重采样更大时间周期 choices=['minutes', 'daily', 'weekly', 'monthly']
G_DT_COMPRESSION = 15  # 合成bar的周期数
G_PLOT = False  # 是否绘图,可提供绘图参数:'style="candle"'
G_QUANTSTATS = True  # 是否使用 quantstats 分析测试结果
G_P_LOG_PRINT = False  # 是否输出日志到控制台
G_P_LOG_FILE = True  # 是否输出日志到文件

G_OPTS = False  # 是否参数调优
G_P_RPP = [10, True, 2, 10, 1]  # 参数[默认值,是否优化最小值,最大值,步长]
G_P_SPP = [19, False, 2, 10, 1]  # 参数[默认值,是否优化最小值,最大值,步长]
G_P_RSP = [5, False, 2, 10, 1]  # 参数[默认值,是否优化最小值,最大值,步长]
G_P_OJK = [1, False, 1, 100, 20]  # 参数[默认值,是否优化最小值,最大值,步长]
G_P_KEY = [False, 4100, 3400, 1800, 6000]  # 关键价格[是否启用, 价格1, 价格2]
G_P_PARAM = {
    'rpp': (range(G_P_RPP[2], G_P_RPP[3], G_P_RPP[4]) if G_OPTS and G_P_RPP[1] else G_P_RPP[0]),
    'spp': (range(G_P_SPP[2], G_P_SPP[3], G_P_SPP[4]) if G_OPTS and G_P_SPP[1] else G_P_SPP[0]),
    # 'rsp': (range(G_P_RSP[2], G_P_RSP[3], G_P_RSP[4]) if G_OPTS and G_P_RSP[1] else G_P_RSP[0]),
    # 'ojk': (range(G_P_OJK[2], G_P_OJK[3], G_P_OJK[4]) if G_OPTS and G_P_OJK[1] else G_P_OJK[0]),
    # 'key': G_P_KEY[1:] if G_P_KEY[0] else None
}
G_OPTS_RESULT = True  # 是否使用上次优化结果

"""-------主函数---------"""
if __name__ == '__main__':
    runstrat()

NameError: name 'runstrat' is not defined

In [10]:
result_one = G_RESULTS_OPT[4]
parm = '{:}{:}'.format(G_P_RPP[2:], G_P_SPP[2:])
print(parm)
print(G_RESULTS_OPT.index(result_one))
print(result_one[0].p.rpp, ' ', result_one[0].p.spp)
str(G_P_PARAM)
# type(G_P_PARAM)
print('格式化金额千分位分隔符: %s' % format(1000000, ',.2f'))

[2, 10, 1][2, 10, 1]
4
2   6
格式化金额千分位分隔符: 1,000,000.00


In [56]:
import pandas as pd

df = pd.read_csv(filepath_or_buffer='./datas/results_opt.csv',
                 sep='\t')
print(df[df.columns[:11]].info)

<bound method DataFrame.info of     id  rpp  spp   rtot%  won%  rnorm%  maxDD%  sharpe     comm%    total    pnl_net
0    9 1.00 9.00  211.36 18.18    0.00   98.97    0.55     12.09   232.00 148,382.00
1    6 1.00 6.00  204.71 15.41    0.00   59.35    0.48      7.20   345.00 287,611.00
2    4 1.00 4.00  177.70 17.70    0.00   64.34    0.53     14.63   600.00 263,193.00
3   18 2.00 9.00  170.69 15.49    0.00   89.55    0.53     14.13   227.00  95,332.00
4   40 5.00 4.00  142.78 20.64    0.00   39.32    0.49      6.29   437.00 171,500.00
5    7 1.00 7.00  140.10 15.71    0.00   65.83    0.46     13.87   313.00 112,433.00
6   15 2.00 6.00  135.68 12.86    0.00  114.85    0.43     37.69   740.00  77,476.00
7   12 2.00 3.00  124.50 15.67    0.00   67.49    0.53     17.22   454.00  45,928.00
8   76 9.00 4.00  119.03 26.67    0.00   32.68    0.55      4.77   361.00 131,852.00
9   67 8.00 4.00  115.88 24.54    0.00   33.82    0.59      5.84   384.00 121,656.00
10  65 8.00 2.00  113.12 19.53   

In [448]:
df = df.sort_values('rtot%')
pd.set_option('display.min_rows', 300)  # 确定显示的最小行有多少
print(df.loc[:, :'rtot%'].head(100))

    id  rpp   spp  rtot%
89  89 3.00 12.00 -28.01
88  88 5.00 17.00 -28.01
87  87 5.00 16.00 -27.85
86  86 4.00 15.00 -27.16
85  85 2.00 12.00 -26.99
84  84 1.00 12.00 -26.46
83  83 5.00 19.00 -24.74
82  82 5.00 18.00 -24.74
81  81 5.00 15.00 -24.64
80  80 6.00 19.00 -20.25
79  79 3.00 19.00 -19.53
78  78 3.00 11.00 -19.51
77  77 3.00 10.00 -19.28
76  76 6.00 12.00 -16.98
75  75 8.00 12.00 -16.33
74  74 4.00 17.00 -15.95
73  73 4.00 16.00 -15.83
72  72 6.00 18.00 -15.79
71  71 1.00 13.00 -15.30
70  70 8.00 13.00 -15.18
69  69 6.00 13.00 -14.89
68  68 7.00 12.00 -14.28
67  67 8.00 14.00 -13.73
65  65 8.00 19.00 -13.46
66  66 8.00 18.00 -13.46
64  64 9.00 12.00 -13.32
63  63 8.00 17.00 -13.27
62  62 7.00 19.00 -12.72
61  61 7.00 18.00 -12.72
60  60 7.00 17.00 -12.44
59  59 7.00 13.00 -12.29
58  58 9.00 13.00 -12.21
57  57 1.00 10.00 -11.92
56  56 9.00 17.00 -11.89
55  55 9.00 16.00 -11.80
54  54 7.00 14.00 -10.09
52  52 4.00 19.00  -9.93
53  53 4.00 18.00  -9.93
51  51 6.00 14.00  -9.61


In [23]:
G_P_RPP = [7, True, 2, 10, 1]  # 参数[默认值,是否优化最小值,最大值,步长]
G_P_SPP = [4, True, 2, 10, 1]  # 参数[默认值,是否优化最小值,最大值,步长]
G_P_OJK = [1, True, 1, 3, 1]  # 参数[默认值,是否优化最小值,最大值,步长]
G_P_PARAM = {
    'rpp': (range(G_P_RPP[2], G_P_RPP[3], G_P_RPP[4]) if G_P_RPP[1] else G_P_RPP[0]),
    'spp': (range(G_P_SPP[2], G_P_SPP[3], G_P_SPP[4]) if G_P_SPP[1] else G_P_SPP[0]),
    'ojk': (range(G_P_OJK[2], G_P_OJK[3], G_P_OJK[4]) if G_P_OJK[1] else G_P_OJK[0]),
}
print(' param:{:}'.format(str(G_P_PARAM).replace(' ', '')))

filename = (('{:}_{:}_{:}-{:}_{:}' + ('_rs_opt.csv' if G_OPTS_RESULT else '_opt.csv'))
            .format(G_FILE_PATH[:12], (str(G_DT_COMPRESSION) + (G_DT_TIMEFRAME[:1])), G_DT_START, G_DT_END, (str(list(G_P_PARAM.values())).replace(' ', '').replace(':', '')), ))
# filename = 'datas\\SQRB13-5m_2013-01-04-2014-04-02_range(2,10),range(2,10),range(1,3)_rs_opt.csv'
print(filename)  # 打印文件路径
# res_df.to_csv(filename, sep='\t', float_format='%.2f')  # 保存分析数据到文件

 param:{'rpp':range(2,10),'spp':range(2,10),'ojk':range(1,3)}
datas\SQRB13-15m_2013-01-04-2022-04-02_[range(2,10),range(2,10),range(1,3)]_rs_opt.csv


In [422]:
global G_RESULTS_OPT, G_COUNT  # 申明要使用全局变量
results_opt = G_RESULTS_OPT.copy()

In [384]:

for l1 in results_opt[:2]:
    print(list(l1)[0].analyzers.returns.rets)
for l2 in l1:
    # print(dir(l2))
    rets = l2.analyzers.returns.rets
print(' {:}, {:} '.format(l2.p.rpp, l2.p.spp)
      + ' {:.4f} '.format((l2.analyzers.returns.rets.get('rtot')))
      + ' {:} '.format(list(rets.keys())[0])
      + ' {:.4f} '.format(list(rets.values())[0])
      )

OrderedDict([('rtot', 0.20507733558250632), ('ravg', 5.159178253647958e-05), ('rnorm', 5.159311341537959e-05), ('rnorm100', 0.005159311341537959)])
 1, 10  0.2051  rtot  0.2051 
OrderedDict([('rtot', 0.0673670171684912), ('ravg', 1.6947677275092124e-05), ('rnorm', 1.6947820887785934e-05), ('rnorm100', 0.0016947820887785933)])
 1, 11  0.0674  rtot  0.0674 


In [20]:
G_P_RPP
(str(G_DT_COMPRESSION) + (G_DT_TIMEFRAME[:1]))


'15m'